In [2]:
%pwd
%cd /home/arno/Projects/EEGDecodingTest/My

/home/arno/Projects/EEGDecodingTest/My


/home/arno/conda/envs/MNE-EEG/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pickle
from tqdm import tqdm

/home/arno/conda/envs/MNE-EEG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name ='/home/arno/Projects/EEGDecodingTest/My/LLM/Qwen2.5-7B-Instruct'
device="cuda"
    
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def getEmbedding(text, raw=False):
        if not raw:
                messages = [
                        # {"role": "system", "content": ' You are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.'},
                        {"role": "system", "content": ' You are a helpful assistant.'},
                        {"role": "user", "content": text}
                        ]
                text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        return model.get_input_embeddings()(model_inputs.input_ids)



Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.79it/s]


In [27]:
#region 已经训练过一个模型从EEG latent预测embedding，读取所有的（character，训练embed）
import re

sub="10"

root_dir = 'Data/qwen-characterSplit'

eegEmbedingFile="/home/arno/Projects/EEGDecodingTest/My/Data/yhw/subSplit/3subjects/latent_to_embed_model_sub04_sub05_sub08_predictions.pkl"

characters=[]
for sub_dir in os.listdir(root_dir):
    # if "05" in sub_dir:
    if "04" in sub_dir or "05" in sub_dir or "08" in sub_dir or True:
        sub_dir_path = os.path.join(root_dir, sub_dir)
        if os.path.isdir(sub_dir_path):
            for file_name in os.listdir(sub_dir_path):
                if file_name.endswith('.pkl') and 'latent' in file_name and '_tokenid'  in file_name and'_tokenEmbedding' in file_name:
                    file_path = os.path.join(sub_dir_path, file_name)
                    with open(file_path, 'rb') as file:
                        loaded_data = pickle.load(file)
                    # 如果loaded_data里面每一个元素没有下面那么多成分，以每个的shape为准，shape总是对的。比如没有一个3584长度的东西说明embedding没在里面
                    for  eeg, character, latent, tokenid, embed in loaded_data:
                        # eeg.shape (1, 128, 256) 多通道EEG信号
                        # character -> str 一个中文汉字
                        # latent.shape (1, 64) EEG压缩后的向量
                        # tokenid array([24]) 汉字对应的token
                        # embed.shape (3584,)  词嵌入向量
                        characters.append([character,embed])
                        
                    
with open(eegEmbedingFile, 'rb') as file:
    loaded_data = pickle.load(file)


#endregion

In [28]:
len(characters)

185828

In [25]:

# 输出文件名
output_file = "responses-sub04_sub05_sub08.txt"
# output_file = f"responses-sub{sub}.txt"

sysprompt="You are a helpful assistant. 用户向你诉说了一些话语，但是你没有听得特别清楚。不需要考虑用户说的话的语义，尝试忠实地复述用户的话。回答只需要包含用户的话，不要有其他的内容。"
# 一个字一个字重建


jumpTo=0
batch_size = 100  # 设置批量大小
batch_characters = []  # 用于存储当前批次的字符
batch_gt_embeds = []  # 用于存储当前批次的嵌入

with open(output_file, "w") as file:
    for ch, gt_embed in tqdm(characters, desc="Processing characters"):
        jumpTo -= 1
        if jumpTo > 0:
            continue

        # 转换 gt_embed 为 Tensor 并添加维度
        gt_embed = torch.tensor(gt_embed)
        if gt_embed.shape[0] == 3584:
            gt_embed = gt_embed.unsqueeze(0)
        insert_message = gt_embed.unsqueeze(0).to(torch.bfloat16).to(device)

        # 将当前字符和嵌入加入批量
        batch_characters.append(ch)
        if insert_message.shape[1]!=1:
            insert_message = insert_message[:,0,:].unsqueeze(1)
        batch_gt_embeds.append(insert_message)

        # 如果达到批量大小，进行推理
        if len(batch_characters) == batch_size:
            # 构建批量输入
            batched_input_embeds = torch.cat([
                getEmbedding(f'<|im_start|>system\n{sysprompt}<|im_end|>\n<|im_start|>user\n', raw=True).repeat(batch_size, 1, 1),
                torch.cat(batch_gt_embeds, dim=0),
                getEmbedding('<|im_end|>\n<|im_start|>assistant\n', raw=True).repeat(batch_size, 1, 1)
            ], dim=1)

            # 使用模型生成结果
            predicted_ids = model.generate(
                inputs_embeds=batched_input_embeds,
                max_new_tokens=512,
                do_sample=True
            )

            # 解码并写入结果
            responses = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
            for ch, response in zip(batch_characters, responses):
                file.write(f"{ch}: {response}\n")
                file.flush()  # 确保及时写入磁盘

            # 清空当前批量
            batch_characters = []
            batch_gt_embeds = []

    # 处理最后一批未完成的样本
    if batch_characters:
        batched_input_embeds = torch.cat([
            getEmbedding(f'<|im_start|>system\n{sysprompt}<|im_end|>\n<|im_start|>user\n', raw=True).repeat(len(batch_characters), 1, 1),
            torch.cat(batch_gt_embeds, dim=0),
            getEmbedding('<|im_end|>\n<|im_start|>assistant\n', raw=True).repeat(len(batch_characters), 1, 1)
        ], dim=1)

        predicted_ids = model.generate(
            inputs_embeds=batched_input_embeds,
            max_new_tokens=512,
            do_sample=True
        )

        responses = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
        for ch, response in zip(batch_characters, responses):
            file.write(f"{ch}: {response}\n")
            file.flush()

print("Processing completed. Responses saved to", output_file)


Processing characters:  84%|████████▍ | 56099/66972 [15:09<02:56, 61.71it/s]


KeyboardInterrupt: 

## Evaluate

In [26]:
# 初始化统计指标
total_lines = 0
matching_count = 0
non_matching_count = 0

chs, responses=[],[]
# 打开文件并逐行读取
with open(output_file, "r") as file:
    for line in file:
        # 去掉每行的首尾空格并分割 'ch' 和 'response'
        parts = line.strip().split(": ")
        
        # 确保格式正确
        if len(parts) == 2:
            ch, response = parts
            chs.append(ch)
            total_lines += 1
            
            # 比较 ch 和 response
            if ch == response:
                matching_count += 1
                responses.append(response)
            else:
                non_matching_count += 1
                responses.append("？")

# 计算统计指标
match_percentage = (matching_count / total_lines) * 100 if total_lines > 0 else 0
non_match_percentage = (non_matching_count / total_lines) * 100 if total_lines > 0 else 0


# print("".join(chs))
# print("="*20)
# print("".join(responses))
# 输出统计结果
print(f"Total lines: {total_lines}")
print(f"Matching count: {matching_count}")
print(f"Non-matching count: {non_matching_count}")
print(f"Matching percentage: {match_percentage:.2f}%")
print(f"Non-matching percentage: {non_match_percentage:.2f}%")


Total lines: 56000
Matching count: 55480
Non-matching count: 520
Matching percentage: 99.07%
Non-matching percentage: 0.93%


sub 04

Total lines: 22324
Matching count: 22114
Non-matching count: 210
Matching percentage: 99.06%
Non-matching percentage: 0.94%

sub 05

Total lines: 22324
Matching count: 22111
Non-matching count: 213
Matching percentage: 99.05%
Non-matching percentage: 0.95%

sub 06



sub 07



sub 05



sub 10

Total lines: 22324
Matching count: 22106
Non-matching count: 218
Matching percentage: 99.02%
Non-matching percentage: 0.98%

responses-sub04_sub05_sub06

Total lines: 66972
Matching count: 66356
Non-matching count: 616
Matching percentage: 99.08%
Non-matching percentage: 0.92%

sub04_sub05_sub07

Total lines: 66972
Matching count: 66344
Non-matching count: 628
Matching percentage: 99.06%
Non-matching percentage: 0.94%

sub04_sub05_sub08

Matching percentage: 99.07%
Non-matching percentage: 0.93%